## Supporting Code for Verifying the Overlay
(Most Code is pulled from the Software implementation)

In [1]:
import random
import math

def gcd(a, b):
    while b != 0:
        a, b = b, a % b
    return a
# Function to generate public key
def pubKeyGen(p, q):
    #define phi with (p,q)
    phi = (p-1)*(q-1)
    # calculate/return a value for e (public key)
    e = 2
    while (e < phi):
        if(gcd(e, phi) == 1):
            break
        else:
            e=e+1
    return e

# Functions to generate public/private keys
def generate_keypair(prime1, prime2):

    n = prime1 * prime2
    phi = (prime1 - 1) * (prime2 - 1)

    # Generate a public key value e from the primes
    e = pubKeyGen(prime1, prime2)

    # Compute the private key d.
    d = pow(e, -1, phi)
 
    # Return the public and private keypairs
    return ((e, n), (d, n))

# Function to convert the message to an array of integers.
def prepInput(message):
    intArrString = [ord(char) for char in message]
    return intArrString
# Function to reverse the action above and return a string from an array of integers.
def convOutput(orgMsg):
    stringFrInt = [chr(char) for char in orgMsg]
    return ''.join(stringFrInt)

def encrypt(message, public_key):

    # Unpack the key into its components
    e, n = public_key
    testWord = [ord(char) for char in message]
    print(testWord)
    # Convert each letter in the plaintext to numbers based on the character using a^b mod m
    cipher = [pow(ord(char), e, n) for char in message]
     
    # Return the array of bytes
    return cipher

 

def decrypt(cipher, private_key):
    # Unpack the key into its components
    d, n = private_key

    # Generate the plaintext based on the ciphertext and key using a^b mod m
    message = [chr(pow(char, d, n)) for char in cipher]

    # Return the array of bytes as a string
    return ''.join(message)


In [2]:
# Test of converting string to a list of int - Passed!
inStr = "Hello"
inStr = prepInput(inStr)
print(inStr)
inStr =convOutput(inStr)
print(inStr)

[72, 101, 108, 108, 111]
Hello


In [3]:
# Example test of Encryption/Decryption with Same Keys as HW
msg = "Hello, World!"
public_key = (65537, 6103297) # Public key used for HW with primes 3011 and 2027
private_key = (2416153, 6103297) # Private key used for HW
encryptArr = encrypt(msg, public_key)
print("Encrypted Values: ", encryptArr) # Encrypted values should match outcome of overlay below
decryptArr = decrypt(encryptArr, private_key)
print("Private Key: ", private_key)
print("Decrypted Values: ", decryptArr) # Decrypted value should return the original message

[72, 101, 108, 108, 111, 44, 32, 87, 111, 114, 108, 100, 33]
Encrypted Values:  [2374081, 634949, 683399, 683399, 4222941, 5390756, 6054897, 2047352, 4222941, 4107156, 683399, 3812635, 469162]
Private Key:  (2416153, 6103297)
Decrypted Values:  Hello, World!


# Overlay Code

In [19]:
# Implementation Framework for AXI Stream (adapted from HW4 until Bitstream is working)

from pynq import Overlay
# Bit file from Vivado
ol = Overlay('/home/xilinx/pynq/overlays/dma_axis_ip_example.bit')
dma = ol.axi_dma
dma_send = ol.axi_dma.sendchannel
dma_recv = ol.axi_dma.recvchannel

hls_ip = ol.encrypt_0 

CONTROL_REGISTER = 0x00
hls_ip.write(CONTROL_REGISTER, 0x81)


In [20]:
from pynq import allocate
import numpy as np

def runKernel(message):
# Send to PL for encryption/decryption
    input_buffer = allocate(shape=(len(message),), dtype=np.uint32)
    input_buffer[:] = [ord(char) for char in message]
    dma_send.transfer(input_buffer)
    output_buffer = allocate(shape=(len(message),), dtype=np.uint32)
    dma_recv.transfer(output_buffer)
    dma_send.wait()
    dma_recv.wait()
    
    return output_buffer

In [21]:
import time
# runKernel takes the message you want to encrypt and returns the output_buffer with the encrypted results.
message1 = "Hello, World!"
start = time.time()
output_buffer = runKernel(message1)
end = time.time()
print(output_buffer) # Printed output should match the values from the SW test above.
print("Duration: ", end - start)

[2374081  634949  683399  683399 4222941 5390756 6054897 2047352 4222941
 4107156  683399 3812635  469162]
Duration:  0.016942739486694336


In [99]:
# Run decryption on the SW function by converting output_buffer to list and perform decrypt
cipher_out = output_buffer.tolist()
decrypt(cipher_out, private_key)

'Hello, World!'

## Testing the Encryption with Key Generation Overlay

In [22]:
from pynq import Overlay
# Bit file from Vivado
ol = Overlay('/home/xilinx/pynq/overlays/RSA2/dma_axis_ip_example.bit')
dma = ol.axi_dma
dma_send = ol.axi_dma.sendchannel
dma_recv = ol.axi_dma.recvchannel

hls_ip = ol.encrypt_0 

CONTROL_REGISTER = 0x00
hls_ip.write(CONTROL_REGISTER, 0x81)


In [23]:
hls_ip.register_map

RegisterMap {
  prime1 = Register(prime1=write-only),
  prime2 = Register(prime2=write-only)
}

In [40]:
hls_ip.register_map.prime1 = 2027
hls_ip.register_map.prime2 = 3011



In [48]:
from pynq import allocate
import numpy as np

def runKernel(message):
# Send to PL for encryption/decryption
    input_buffer = allocate(shape=(len(message),), dtype=np.uint32)
    input_buffer[:] = [ord(char) for char in message]
    dma_send.transfer(input_buffer)
    output_buffer[:] = allocate(shape=(len(message),), dtype=np.uint32)
    dma_recv.transfer(output_buffer)
    dma_send.wait()
    dma_recv.wait()
    
    return output_buffer

In [73]:
import time
msg = "Hello, World"
# Get the encrypted Array and pass to runKernel to test
public_key = (65537, 6103297)
encrArr = encrypt(msg, public_key)
print(encrArr)

start = time.time()
output = runKernel(msg)

end = time.time()

print(output)
print("Duration: ", end - start)
# if results match, the overlay works - PASSED

[72, 101, 108, 108, 111, 44, 32, 87, 111, 114, 108, 100]
[2374081, 634949, 683399, 683399, 4222941, 5390756, 6054897, 2047352, 4222941, 4107156, 683399, 3812635]
[2374081  634949  683399  683399 4222941 5390756 6054897 2047352 4222941
 4107156  683399 3812635]
Duration:  0.019195079803466797


## Overlay with Encrypt and Decrypt Functionality
Added a boolean axi stream variable to determine if the function is encrypting or decrypting

In [68]:
from pynq import Overlay
# Bit file from Vivado
ol = Overlay('/home/xilinx/pynq/overlays/RSA3/dma_axis_ip_example.bit')
dma = ol.axi_dma
dma_send = ol.axi_dma.sendchannel
dma_recv = ol.axi_dma.recvchannel

hls_ip = ol.encrypt_0 

CONTROL_REGISTER = 0x00
hls_ip.write(CONTROL_REGISTER, 0x81)


In [69]:
hls_ip.register_map

RegisterMap {
  prime1 = Register(prime1=write-only),
  prime2 = Register(prime2=write-only),
  ende = Register(ende=write-only, RESERVED=write-only)
}

In [70]:
hls_ip.register_map.prime2 = 3011
hls_ip.register_map.prime1 = 2027
hls_ip.register_map.ende = 0 #Set to encrypt now



In [74]:
from pynq import allocate
import numpy as np

def runEncrypt(message):
    hls_ip.register_map.ende = 0 #Set to encrypt now
# Send to PL for encryption/decryption
    input_buffer = allocate(shape=(len(message),), dtype=np.uint32)
    input_buffer[:] = [ord(char) for char in message]
    dma_send.transfer(input_buffer)
    output_buffer[:] = allocate(shape=(len(message),), dtype=np.uint32)
    dma_recv.transfer(output_buffer)
    dma_send.wait()
    dma_recv.wait()
    
    return output_buffer

In [83]:
def runDecrypt(cipher_out):
    hls_ip.register_map.ende = 1 #Set to decrypt now
# Send to PL for encryption/decryption
    input_buffer = allocate(shape=(len(cipher_out),), dtype=np.uint32)
    input_buffer[:] = [x for x in cipher_out]
    dma_send.transfer(input_buffer)
    output_buffer[:] = allocate(shape=(len(cipher_out),), dtype=np.uint32)
    dma_recv.transfer(output_buffer)
    dma_send.wait()
    dma_recv.wait()
    
    return output_buffer

In [94]:
import time
msg = "Hello, World"
# Get the encrypted Array and pass to runKernel to test
public_key = (65537, 6103297)
encrArr = encrypt(msg, public_key)
print(encrArr)

start = time.time()
output = runEncrypt(msg)

end = time.time()

print(output)
print("Duration: ", end - start)
# if results match, the overlay works - PASSED


[72, 101, 108, 108, 111, 44, 32, 87, 111, 114, 108, 100]
[2374081, 634949, 683399, 683399, 4222941, 5390756, 6054897, 2047352, 4222941, 4107156, 683399, 3812635]
[2374081  634949  683399  683399 4222941 5390756 6054897 2047352 4222941
 4107156  683399 3812635]
Duration:  0.012873649597167969


In [97]:
import time
msg = "Hello, World"
# Get the encrypted Array and pass to runKernel to test
public_key = (2416153, 6103297)
decrArr = decrypt(encrArr, private_key)
print(decrArr)

start = time.time()
plain = runDecrypt(output)

end = time.time()

print(plain)
print("Duration: ", end - start)
# if results match, the overlay works - PASSED

Hello, World
[ 72 101 108 108 111  44  32  87 111 114 108 100]
Duration:  0.7139289379119873
